In [ ]:
#!/usr/bin/env python3

%matplotlib qt
import numpy as np
import pandas as pd
# import torch
# from dgl.geometry import farthest_point_sampler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
df = pd.read_csv('/home/julius/Desktop/viewpoint_sampling/misc/poses_6d.csv')
#print(df.head())

# min = df['z'].abs().max()
# min_idx = df.index[df['z'] == min]
# print(min)
# print(min_idx.tolist())
# print(df.loc[min_idx.tolist()])


samples = 3

In [ ]:
# x = torch.from_numpy(df.values)
# x = x.unsqueeze(0) 
# # print(x.shape)
# # print(type(x[0]))

# # print(x.shape)
# # y = torch.rand((2, 10, 3))
# # print(y)
# # print(y.shape)
# point_idx = farthest_point_sampler(x, samples)#, start_idx=0)
# point_idx = point_idx.flatten().tolist()
# print(point_idx)

# farthest_points = df.loc[point_idx]
# #farthest_points.sample(2)
# print(farthest_points['x'].values)


In [ ]:
def fps(points, n_samples):
    """
    points: [idx, 3] array containing the whole point cloud
    n_samples: samples you want in the sampled point cloud typically << idx 
    """
    points = np.array(points)
    
    # Represent the points by their indices in points
    points_left = np.arange(len(points)) # [scipy_points]

    # Initialise an array for the sampled indices
    sample_inds = np.zeros(n_samples, dtype='int') # [S]

    # Initialise distances to inf
    dists = np.ones_like(points_left) * float('inf') # [scipy_points]

    # Select a point from points by its index, save it
    selected = 216
    sample_inds[0] = points_left[selected]

    # Delete selected 
    points_left = np.delete(points_left, selected) # [scipy_points - 1]

    # Iteratively select points for a maximum of n_samples
    for i in range(1, n_samples):
        # Find the distance to the last added point in selected
        # and all the others
        last_added = sample_inds[i-1]
        
        dist_to_last_added_point = (
            (points[last_added] - points[points_left])**2).sum(-1) # [scipy_points - i]

        # If closer, updated distances
        dists[points_left] = np.minimum(dist_to_last_added_point, 
                                        dists[points_left]) # [scipy_points - i]

        # We want to pick the one that has the largest nearest neighbour
        # distance to the sampled points
        selected = np.argmax(dists[points_left])
        sample_inds[i] = points_left[selected]

        # Update points_left
        points_left = np.delete(points_left, selected)

    return points[sample_inds]


In [ ]:
values = df[['x', 'y', 'z']].to_numpy()
pnts = fps(values, samples)


In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist

p = samples
# idx = 16000000

# # Find a convex hull in O(idx log idx)
# points = np.random.rand(idx, 3)   # idx random points in 3-D
points = values
# # Returned 420 points in testing
# hull = ConvexHull(points)

# # Extract the points forming the hull
# hullpoints = points[hull.vertices,:]
hullpoints = values
# Naive way of finding the best pair in O(H^2) time if H is number of points on
# hull
hdist = cdist(hullpoints, hullpoints, metric='euclidean')
# print(hdist)
# print(hdist.shape)
# Get the farthest apart points
bestpair = np.unravel_index(hdist.argmax(), hdist.shape)

#print(bestpair.shape)

P = np.array([hullpoints[bestpair[0]],hullpoints[bestpair[1]]])

print("P:", P)
# Now we have a problem
print("Finding optimal set...")
while len(P)<p:
  print("P size = {0}".format(len(P)))
  distance_to_P = cdist(points, P, metric='euclidean')
  #print("distance_to_P", distance_to_P)
  print(distance_to_P.shape)
  minimum_to_each_of_P = np.min(distance_to_P, axis=1)
  best_new_point_idx   = np.argmax(minimum_to_each_of_P)
  best_new_point = np.expand_dims(points[best_new_point_idx,:],0)
  P = np.append(P,best_new_point,axis=0)


In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(projection='3d')

ax.scatter(df['x'].values, df['y'].values, df['z'].values,  color='r', s=3, marker='X')
#ax.plot(farthest_points['x'].values, farthest_points['y'].values, farthest_points['z'].values, marker='o', markersize=5, color='black', label='Torch-Method')
#ax.scatter(P[:,0], P[:,1], P[:,2], marker='X', s=40, color='green', label='Scipy-Method')
ax.scatter(pnts[:,0], pnts[:,1], pnts[:,2], marker='*', s=40, color='blue', label=str(samples)+' samples')


#ax.plot(total_points[:, 0], total_points[:, 1], total_points[:, 2])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
ax.set_title("Farthest Point Sampling of Viewing Hemisphere")
plt.show()

In [ ]:
rows_list = []
idx_list = np.array([])


for idx, subarr in enumerate(pnts):
    rows_list.append((df.loc[(df['x'] == subarr[0]) & (df['y'] == subarr[1]) & (df['z'] == subarr[2])]).values.flatten())
    idx_list = np.append(idx_list, ((df.index[(df['x'] == subarr[0]) & (df['y'] == subarr[1]) & (df['z'] == subarr[2])]).to_numpy())) 
    idx_list = idx_list.astype(int)
new_df = pd.DataFrame(columns= df.columns, data=rows_list, index=idx_list)
new_df.head(100)
#print(df.to_string())


In [ ]:
# Import the necessary libraries
from skimage.io import imread, imread_collection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import numpy as np
import os

# Read the image as a float
folder = '/home/julius/Documents/Julius_03_depth_masks/001_standing_coated'
img_list = []
for img in sorted(os.listdir(folder)):
    
    # image = imread(os.path.join(folder, img))
    image = (os.path.join(folder, img))
    img_list.append(image)
    # image = imread('/home/julius/Documents/Julius_03_depth_masks/001_standing_coated/mask_001.png')
    # print("image shape: ", image.shape)
    # print("image max: ", np.max(image))
    # print("image min: ", np.min(image))
    # r = image[:, :,0]
    # g = image[:, :,1]
    # b = image[:, :,2]

    # r_unique, ru_idx = np.unique(r, return_index=True)
    # g_unique, gu_idx = np.unique(g, return_index=True)
    # b_unique, bu_idx = np.unique(b, return_index=True)

    #ru_idx = np.unravel_index(ru_idx, image.shape)

    # unique_colors = np.unique(image.reshape(-1, image.shape[2]), axis=0) #np.column_stack((r_unique, g_unique, b_unique))
    # print(unique_colors)
    # print(image.shape )
    # print(np.unique(image))
    # print("")

    # background = np.argwhere(np.all(image == unique_colors[0], axis=-1))
    # obj1 = np.argwhere(np.all(image == unique_colors[1], axis=-1))
    # obj2 = np.argwhere(np.all(image == unique_colors[2], axis=-1))
    # obj3 = np.argwhere(np.all(image == unique_colors[3], axis=-1))

    #image = image - obj1# print(len(np.ndarray.flatten(indices)))


    # ax = plt.subplot()
    # im = ax.imshow(image, cmap="tab20")

    # # create an Axes on the right side of ax. The width of cax will be 5%
    # # of ax and the padding between cax and ax will be fixed at 0.05 inch.
    # divider = make_axes_locatable(ax)
    # cax = divider.append_axes("right", size="5%", pad=0.05)
    # plt.title(img)
    # plt.colorbar(im, cax=cax)

    # plt.show()



In [22]:

from skimage.io import imread, imread_collection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import numpy as np
import os

# Read the image as a float
folder_per_obj_masks = '/home/julius/Documents/Julius_03_masks'
folder_full_blender_depth = '/home/julius/Documents/Julius_03_depth_masks'
scenes = sorted(os.listdir(folder_per_obj_masks))

canister_list = []
small_bottle_list = []
medium_bottle_list = []
large_bottle_list = []
full_depth_list = []

##here implement for loop over all scenes
for scene in [scenes[0]]:
    folder_to_evaluate = f'/home/julius/Documents/Julius_03_auswertung/scenes_trained_with_other_params/{scene}/nerfacto/depth_nerf_sigma_10_iter_5000/depth_nerf'
    print(folder_to_evaluate)
    folder1=os.path.join(folder_per_obj_masks, scene)
    for img in sorted(os.listdir(folder1)):
        image = (os.path.join(folder1, img))
        if "canister" in img:
            canister_list.append(image)
        elif "small" in img:
            small_bottle_list.append(image)
        elif "medium" in img:
            medium_bottle_list.append(image)
        elif "large" in img:
            large_bottle_list.append(image)

    canister_masks = imread_collection(canister_list)
    small_bottle_masks = imread_collection(small_bottle_list)
    medium_bottle_masks = imread_collection(medium_bottle_list)
    large_bottle_masks = imread_collection(large_bottle_list)

    folder2=os.path.join(folder_full_blender_depth, scene)
    for img in sorted(os.listdir(folder2)):
        image = (os.path.join(folder2, img))
        full_depth_list.append(image)

    blender_depth = imread_collection(full_depth_list)

    if len(canister_masks) != len(blender_depth):
        raise ValueError (f"Lists do not have the same size! Check folder {folder1} or {folder2}!")
    else:
        for idx, item in enumerate(blender_depth):
           
            # canister_coords = np.argwhere(canister_masks[idx][:,:] > 0)
            # small_bottle_coords = np.argwhere(small_bottle_masks[idx][:,:] > 0)
            # medium_bottle_coords = np.argwhere(medium_bottle_masks[idx][:,:] > 0)
            # large_bottle_coords = np.argwhere(large_bottle_masks[idx][:,:] > 0)

            # all_obj_coords = canister_masks[idx] + small_bottle_masks[idx] + medium_bottle_masks[idx] + large_bottle_masks[idx]
            #plt.imshow(all_obj_coords)
            #print(all_obj_coords.shape)
            #print(len(canister_coords))

            # depth_img = imread(f"/home/julius/Documents/Julius_03_depth_masks/001_standing_coated/depth_{idx:03d}.png")
            # 
            #print(np.argwhere((np.equal(depth_img, all_obj_coords)==False)))
                        
            canister_gt = np.where(canister_masks[idx] > 0, blender_depth[idx], 0)
            small_bottle_gt = np.where(small_bottle_masks[idx] > 0, blender_depth[idx], 0)
            medium_bottle_gt = np.where(medium_bottle_masks[idx] > 0, blender_depth[idx], 0)
            large_bottle_gt = np.where(large_bottle_masks[idx] > 0, blender_depth[idx], 0)
            print(canister_gt[0].shape)
            print(small_bottle_gt[0].shape)
            print(blender_depth[0].shape)
           
            plt.imshow(blender_depth[idx])

            fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))
            ax[0][0].imshow(canister_gt)
            #ax[0][0].plot(448, 424, color="white", marker="o")
            ax[0][0].title.set_text("canister_gt")
            ax[0][1].imshow(small_bottle_gt)
            ax[0][1].title.set_text("Small bottle")
            ax[1][0].imshow(medium_bottle_gt)
            ax[1][0].title.set_text("Medium bottle")
            ax[1][1].imshow(large_bottle_gt)
            ax[1][1].title.set_text("Large bottle")
            plt.tight_layout()
            plt.show()


            # fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))
            # ax[0][0].imshow(canister_masks[idx])
            # ax[0][0].plot(448, 424, color="white", marker="o")
            # ax[0][0].title.set_text("canister_gt")
            # ax[0][1].imshow(small_bottle_masks[idx])
            # ax[0][1].title.set_text("Small bottle")
            # ax[1][0].imshow(medium_bottle_masks[idx])
            # ax[1][0].title.set_text("Medium bottle")
            # ax[1][1].imshow(large_bottle_masks[idx])
            # ax[1][1].title.set_text("Large bottle")
            # plt.tight_layout()
            # plt.show()

            break

    



/home/julius/Documents/Julius_03_auswertung/scenes_trained_with_other_params/001_standing_coated/nerfacto/depth_nerf_sigma_10_iter_5000/depth_nerf


ValueError: Lists do not have the same size! Check folder /home/julius/Documents/Julius_03_masks/001_standing_coated or /home/julius/Documents/Julius_03_depth_masks/001_standing_coated!

In [3]:
# print(canister_coords)
# print(small_bottle_coords)
# print(medium_bottle_coords)
# print(large_bottle_coords)



[[472 519]
 [472 520]
 [472 521]
 ...
 [635 543]
 [635 544]
 [635 545]]
[]
[[368 927]
 [369 909]
 [369 910]
 ...
 [640 861]
 [640 862]
 [640 863]]
[[129 671]
 [129 672]
 [129 673]
 ...
 [503 675]
 [503 676]
 [503 677]]
